In [38]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urlparse

In [39]:
url_base='https://www.hch.gov.tw'
url_work='/?aid=506&pid=0&page_name=list&pageNo=1'
url_full=url_base+url_work_table
g=requests.get(url_full)
soup=BeautifulSoup(g.content, 'html.parser')

# Get pages link

In [42]:
pages_link=[]
for link in soup.find('ul',class_="pagination pagination-sm justify-content-center"):
    if link.find("a"):
        if link.find("a") == -1 or link.text == "第一頁" or link.text == "«" or link.text == "»" or link.text == "最末頁" or link.text == " ":
            pass
        elif link.find('a').has_attr("disabled"):
            #print('skip:',link.text)
            pages_link.append(url_full)
        else:
            no_page_link=url_full.replace("pageNo=1","pageNo=")
            pages_link.append(no_page_link+link.find('a').text)
pages_link

['https://www.hch.gov.tw/?aid=506&pid=0&page_name=list&pageNo=1',
 'https://www.hch.gov.tw/?aid=506&pid=0&page_name=list&pageNo=2',
 'https://www.hch.gov.tw/?aid=506&pid=0&page_name=list&pageNo=3']

# Get wrok table

In [80]:
for work in soup.find_all('table',class_="table table-striped table-hover news_table")[0].find("tbody").find_all("tr"):
    all_td = work.find_all("td")
    #print(all_td)
    title = all_td[3].text
    place = all_td[0].text
    deadline = all_td[4].text
    originization = all_td[2].text
    #print("召聘職稱",title,'院區',place,"截止日期",deadline,"職缺單位",originization)
    print(all_td[5])

<td class="align-middle" data-title="下載" width="30%">
<a href="/public/jobnews/16517443876072.pdf" rel="noopener noreferrer" style="color: blue;" target="_blank" title="新竹臺大分院簡則-院聘醫事檢驗師.pdf (另開新視窗)"><i alt="" aria-hidden="true" class="fas fa-paperclip"></i> 新竹臺大分院簡則-院聘醫事檢驗師</a>
<a href="/public/jobnews/1651744387617.pdf" rel="noopener noreferrer" style="color: blue;" target="_blank" title="1.信封封面.pdf (另開新視窗)"><i alt="" aria-hidden="true" class="fas fa-paperclip"></i> 1.信封封面</a>
<a href="/public/jobnews/16517443876257.pdf" rel="noopener noreferrer" style="color: blue;" target="_blank" title="2.台大醫院新竹臺大分院各類人員甄選登記表及查核授權書.pdf (另開新視窗)"><i alt="" aria-hidden="true" class="fas fa-paperclip"></i> 2.台大醫院新竹臺大分院各類人員甄選登記表及查核授權書</a>
<a href="/public/jobnews/16517443876352.pdf" rel="noopener noreferrer" style="color: blue;" target="_blank" title="3.作業基金現職人員申請院內外補同意書(新竹臺大分院).pdf (另開新視窗)"><i alt="" aria-hidden="true" class="fas fa-paperclip"></i> 3.作業基金現職人員申請院內外補同意書(新竹臺大分院)</a>
</td>
<td class="align-